In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
168,AFG,Asia,Afghanistan,2020-06-26,30175.0,460.0,675.0,36.0,775.142,11.817,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
278,ALB,Europe,Albania,2020-06-26,2192.0,78.0,49.0,2.0,761.693,27.104,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
452,DZA,Africa,Algeria,2020-06-26,12445.0,197.0,878.0,9.0,283.802,4.492,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-25,France,161913.0,0.0,29781.0,0.0,0.0,0.0
0,2020-06-25,China,84735.0,0.0,4651.0,0.0,0.0,0.0
0,2020-06-25,Italy,239658.0,0.0,34689.0,0.0,0.0,0.0
0,2020-06-25,Spain,247403.0,0.0,28346.0,0.0,0.0,0.0
0,2020-06-25,United States,2416102.0,0.0,122916.0,0.0,0.0,0.0
0,2020-06-25,World,9577787.0,0.0,488384.0,0.0,0.0,0.0
0,2020-06-25,United Kingdom,308193.0,0.0,43384.0,0.0,0.0,0.0
0,2020-06-25,Germany,192711.0,0.0,8954.0,0.0,0.0,0.0
0,2020-06-25,Iran,215388.0,0.0,10111.0,0.0,0.0,0.0
0,2020-06-25,Turkey,192837.0,0.0,5050.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-26,France,161913.0,161348.0,29781.0,29752.0,565.0,29.0
0,2020-06-26,China,84735.0,84701.0,4651.0,4641.0,34.0,10.0
0,2020-06-26,Italy,239658.0,239706.0,34689.0,34678.0,-48.0,11.0
0,2020-06-25,Spain,247403.0,247486.0,28346.0,28330.0,-83.0,16.0
0,2020-06-26,United States,2416102.0,2422310.0,122916.0,124416.0,-6208.0,-1500.0
0,2020-06-26,World,9577787.0,9581803.0,488384.0,489182.0,-4016.0,-798.0
0,2020-06-26,United Kingdom,308193.0,307980.0,43384.0,43230.0,213.0,154.0
0,2020-06-26,Germany,192711.0,192556.0,8954.0,8948.0,155.0,6.0
0,2020-06-26,Iran,215388.0,215096.0,10111.0,10130.0,292.0,-19.0
0,2020-06-26,Turkey,192837.0,193115.0,5050.0,5046.0,-278.0,4.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")